# Gofannon + Google Vertex AI: Deploying an Agent with ADK

This notebook demonstrates how to build and deploy a simple conversational agent to Google Cloud's Vertex AI platform using the Google Agent Development Kit (ADK).

**Key Technologies Used:**
*   **Gofannon:** An open-source Python library that provides pre-built tools and agents. In this example, we'll use its `IssLocator` tool to find the current location of the International Space Station.
*   **Google Vertex AI:** A unified machine learning platform on Google Cloud that allows you to build, deploy, and manage ML models and AI applications, including conversational agents through its Agent Engine capabilities.
*   **Google ADK (Agent Development Kit):** A framework designed to simplify the development, testing, and deployment of AI agents. It provides structures for defining agent behavior, tools, and integrating with language models.

**Overall Goal:**
The primary objective is to create an agent that can report the current location of the ISS. We will first define this agent locally, test it, and then deploy it as a scalable service on Vertex AI Agent Engine.

We will be following the general principles outlined in the ADK documentation for agent deployment: [Google ADK Deploy Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/#create-your-agent)

**Prerequisites:**
To run this example successfully, you will need:
1.  A Google Cloud Platform (GCP) Project.
2.  Billing enabled for your GCP Project.

## 1. Install Dependencies

The first step is to install the necessary Python libraries. 
- `google-cloud-aiplatform[adk,agent_engines]>=1.93.0`: This installs the Vertex AI SDK along with extras for the Agent Development Kit (ADK) and Agent Engines, which are required for building and deploying our agent.
- `gofannon`: This installs the Gofannon library, from which we'll use a pre-built tool.

The `--quiet` flag is used to suppress verbose installation output.

In [16]:
!pip install "google-cloud-aiplatform[adk,agent_engines]>=1.93.0" gofannon --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 23.5 MB/s eta 0:00:00


## 2. Configure GCP Project and Location

Before interacting with Google Cloud services, we need to specify our project ID and the desired region for our resources.
- `PROJECT_ID`: Your unique Google Cloud Project ID.
- `LOCATION`: The GCP region where your Vertex AI resources (like the Agent Engine) will be deployed (e.g., `us-central1`).
- `STAGING_BUCKET`: A Google Cloud Storage (GCS) bucket URI. This bucket is used by Vertex AI for temporary storage of artifacts during processes like model training or, in our case, agent deployment. The `vertexai.init()` call will attempt to create this bucket if it doesn't exist. For simplicity, we derive its name from the project ID.

In [17]:
import vertexai

PROJECT_ID = "agent-coe-text"
LOCATION = "us-central1"
STAGING_BUCKET = f"gs://{PROJECT_ID}-storage-bucket"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

## 3. Define the Agent

Here, we define the core logic and capabilities of our agent using the Google ADK framework.

1.  **Import `IssLocator` from Gofannon:** We import the `IssLocator` class, which is a pre-built tool from the Gofannon library designed to fetch the current coordinates of the International Space Station.
2.  **Export to ADK Tool:** The `IssLocator().export_to_adk()` method converts the Gofannon tool into a format compatible with the Google ADK, allowing it to be used by our ADK agent.
3.  **Create `Agent` Instance:** We instantiate `google.adk.agents.Agent` to define our agent. This involves specifying:
    *   `name`: A descriptive name for the agent (e.g., "weather_time_agent", though in this case it's more of an ISS agent).
    *   `model`: The underlying Large Language Model (LLM) that will power the agent's conversational abilities and decision-making for tool use. Here, we use `gemini-2.0-flash`.
    *   `description`: A brief summary of what the agent does.
    *   `instruction`: A more detailed prompt or set of instructions for the LLM, guiding its behavior and how it should interact with users and tools.
    *   `tools`: A list of tools the agent can use. In this case, it's our ADK-compatible `iss_locator`.

In [18]:
import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import Agent

from gofannon.open_notify_space.iss_locator import IssLocator

iss_locator = IssLocator().export_to_adk()

root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description=(
        "Agent to answer questions about the current location of the international space station (ISS)."
    ),
    instruction=(
        "You are a helpful agent who can answer user questions about the international space station (ISS)."
    ),
    tools=[iss_locator],
)

## 4. Create a Local ADK Application

To test our agent locally before deploying it, we wrap it in an `AdkApp` from `vertexai.preview.reasoning_engines`.
This creates a local application environment for the agent.
- `agent=root_agent`: We pass our defined agent to the application.
- `enable_tracing=True`: This enables tracing capabilities, which can be helpful for debugging and understanding the agent's internal operations (e.g., LLM calls, tool invocations).

In [19]:
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

## 5. Create a Local Session

We create a session with our local `AdkApp`. Sessions are used to maintain conversational context and state over multiple turns of interaction with the agent.
- `user_id="u_123"`: A unique identifier for the user interacting with the agent. This helps in managing and segregating session data if multiple users are involved (though less critical for local testing with a single user).

In [20]:
session = app.create_session(user_id="u_123")

### Display Local Session Information

This cell simply outputs the details of the `session` object created in the previous step. This allows us to inspect its properties, such as its ID, app name, user ID, and current state.

In [21]:
session

Session(id='bae1251a-980e-45aa-84ce-7409371128a0', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1747678964.8205023)

### List Local Sessions

The `app.list_sessions()` method can be used to retrieve a list of all active sessions associated with a particular `user_id` for the local `AdkApp`. This demonstrates how one might manage multiple ongoing conversations.

In [22]:
app.list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[Session(id='bae1251a-980e-45aa-84ce-7409371128a0', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1747678964.8205023)])

### Get a Specific Local Session

If you have a session ID, you can retrieve that specific session using `app.get_session()`. This cell retrieves the session we created earlier (`session.id`) and prints both the original and retrieved session objects to show they are identical.

In [23]:
session2 = app.get_session(user_id="u_123", session_id=session.id)
print(session, "\n\n", session2)

id='bae1251a-980e-45aa-84ce-7409371128a0' app_name='default-app-name' user_id='u_123' state={} events=[] last_update_time=1747678964.8205023 

 id='bae1251a-980e-45aa-84ce-7409371128a0' app_name='default-app-name' user_id='u_123' state={} events=[] last_update_time=1747678964.8205023


## 6. Authenticate for GCP Access (If in Colab)

When running this notebook in a Google Colab environment, this cell is crucial for authenticating your user identity. This authentication allows the notebook to interact with Google Cloud Platform services, such as Vertex AI (for deploying the agent) and Google Cloud Storage (for the staging bucket), using your credentials.

If you are running this notebook in a different environment (e.g., a Vertex AI Workbench instance or a local machine with `gcloud` CLI already configured), this specific Colab authentication step might not be necessary or might be handled differently (e.g., through Application Default Credentials).

In [8]:
from google.colab import auth
auth.authenticate_user()

## 7. Query the Local Agent

Now, we interact with our locally running agent by sending it a query.
- `app.stream_query()`: This method sends the `message` to the agent within the specified `session_id` for the given `user_id`.
- **Streaming Response:** The `stream_query` method returns an iterator. As the agent processes the query, it can yield multiple `event` objects. These events can represent intermediate steps, such as function calls (tool usage) or function responses, before the final answer is generated.
- **Tool Use Example:** The query "where is the iss now?" is designed to trigger the `iss_locator` tool. We expect to see an event indicating the LLM decided to call this tool, followed by an event with the tool's output, and finally an event with the LLM's synthesized response to the user.

The loop prints each event as it's received, allowing us to observe the agent's reasoning process.

In [24]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="where is the iss now?",
):
  print(event)


2025-05-19 18:23:05,866 - google_genai.types - WARNING - Warning: there are non-text parts in the response: ['function_call'],returning concatenated text result from text parts,check out the non text parts for full response from model.


{'content': {'parts': [{'function_call': {'id': 'adk-d743d9ad-58c0-4a0e-82b7-86a8473e12c1', 'args': {}, 'name': 'iss_locator'}}], 'role': 'model'}, 'invocation_id': 'e-b35f2878-3801-44d5-9419-f27a15d51298', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'Xw4Bhh14', 'timestamp': 1747678983.500944}


2025-05-19 18:23:07,034 - opentelemetry.sdk.trace.export - WARNING - Already shutdown, dropping span.
2025-05-19 18:23:07,036 - opentelemetry.sdk.trace.export - WARNING - Already shutdown, dropping span.
2025-05-19 18:23:07,038 - opentelemetry.sdk.trace.export - WARNING - Already shutdown, dropping span.


{'content': {'parts': [{'function_response': {'id': 'adk-d743d9ad-58c0-4a0e-82b7-86a8473e12c1', 'name': 'iss_locator', 'response': {'result': '{"iss_position": {"longitude": "-61.3128", "latitude": "5.3606"}, "timestamp": 1747678986, "message": "success"}'}}}], 'role': 'user'}, 'invocation_id': 'e-b35f2878-3801-44d5-9419-f27a15d51298', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'LAMpZY0X', 'timestamp': 1747678987.036379}


2025-05-19 18:23:09,539 - opentelemetry.sdk.trace.export - WARNING - Already shutdown, dropping span.
2025-05-19 18:23:09,540 - opentelemetry.sdk.trace.export - WARNING - Already shutdown, dropping span.
2025-05-19 18:23:09,541 - opentelemetry.sdk.trace.export - WARNING - Already shutdown, dropping span.


{'content': {'parts': [{'text': 'The ISS is currently at Longitude -61.3128, Latitude 5.3606.\n'}], 'role': 'model'}, 'invocation_id': 'e-b35f2878-3801-44d5-9419-f27a15d51298', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'p7rgQOzY', 'timestamp': 1747678987.040193}


### Extract Final Response from Local Agent

After the `stream_query` loop completes, the `event` variable will hold the last event received. Typically, this is the final response from the agent containing the answer in a text part.
This cell extracts and prints that textual response.

In [25]:
print(event['content']['parts'][0]['text'])

The ISS is currently at Longitude -61.3128, Latitude 5.3606.



## 8. Deploy the Agent to Vertex AI Agent Engine

Once we're satisfied with the local testing, we can deploy the agent to Vertex AI Agent Engine. This makes the agent available as a robust, scalable cloud service.

The deployment process involves packaging the agent definition, its dependencies (like Gofannon), and configuring the Vertex AI service. This operation can take a few minutes to complete as it provisions resources and sets up the environment in Google Cloud.

In [27]:
from vertexai import agent_engines

remote_app = agent_engines.create(
    agent_engine=root_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]" ,
        "gofannon"
    ]
)

2025-05-19 18:26:44,490 - vertexai.agent_engines - INFO - Deploying google.adk.agents.Agent as an application.
2025-05-19 18:26:44,497 - vertexai.agent_engines - INFO - Identified the following requirements: {'google-cloud-aiplatform': '1.93.0', 'cloudpickle': '3.1.1', 'pydantic': '2.11.4'}
2025-05-19 18:26:44,499 - vertexai.agent_engines - WARNING - The following requirements are missing: {'cloudpickle', 'pydantic'}
2025-05-19 18:26:44,500 - vertexai.agent_engines - INFO - The following requirements are appended: {'cloudpickle==3.1.1', 'pydantic==2.11.4'}
2025-05-19 18:26:44,501 - vertexai.agent_engines - INFO - The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'gofannon', 'cloudpickle==3.1.1', 'pydantic==2.11.4']
2025-05-19 18:26:46,590 - vertexai.agent_engines - INFO - Using bucket agent-coe-text-storage-bucket
2025-05-19 18:26:47,923 - vertexai.agent_engines - INFO - Wrote to gs://agent-coe-text-storage-bucket/agent_engine/agent_engine.pkl
2025-05-19 18

### Get Deployed Agent Resource Name

After successful deployment, the `remote_app` object (an instance of `AgentEngine`) will contain information about the deployed resource, including its unique resource name on Google Cloud. This name is useful for managing or referencing the deployed agent engine later (e.g., via `gcloud` CLI or other SDK methods).

In [28]:
remote_app.resource_name

'projects/916874015322/locations/us-central1/reasoningEngines/8079396158851514368'

## 9. Interact with the Deployed Agent

Now that the agent is deployed on Vertex AI, we can interact with it similarly to how we interacted with the local version.

### Create a Session with the Deployed Agent
First, we create a new session, but this time using the `remote_app` object that represents our deployed agent engine. We use a different `user_id` (`u_456`) for this remote session to distinguish it from the local test session.

In [29]:
remote_session = remote_app.create_session(user_id="u_456")
remote_session

{'events': [],
 'last_update_time': 1747679442.536944,
 'state': {},
 'id': '256050419585777664',
 'app_name': '8079396158851514368',
 'user_id': 'u_456'}

### Query the Deployed Agent

We send a query to the deployed agent using `remote_app.stream_query()`. This time, we use a slightly more complex query to test its understanding and tool usage: "if the ISS were to fall out of the sky at this moment, what would it hit (ignore angular momentum)".

The agent should:
1.  Understand it needs the current location of the ISS.
2.  Invoke the `iss_locator` tool.
3.  Use the location data from the tool to formulate an answer, likely indicating it's over an ocean or a specific landmass.

Again, we loop through the streamed events and print them.

In [30]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="if the ISS were to fall out of the sky at this moment, what would it hit (ignore angular momentum)",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-39408b83-52f6-4941-abf1-60ceacffa21d', 'args': {}, 'name': 'iss_locator'}}], 'role': 'model'}, 'invocation_id': 'e-a6f5a055-fb19-4458-99bd-ee88f953164a', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'FnyvOYvu', 'timestamp': 1747679445.733044}
{'content': {'parts': [{'function_response': {'id': 'adk-39408b83-52f6-4941-abf1-60ceacffa21d', 'name': 'iss_locator', 'response': {'result': '{"iss_position": {"longitude": "-42.7579", "latitude": "27.8731"}, "timestamp": 1747679447, "message": "success"}'}}}], 'role': 'user'}, 'invocation_id': 'e-a6f5a055-fb19-4458-99bd-ee88f953164a', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'P47J5XbJ', 'timestamp': 1747679447.59757}
{'content': {'parts': [{'text': 'The ISS is currently over the ocean, at latitude 27.8731 and 

### Extract Final Response from Deployed Agent

Similar to the local test, this cell extracts and prints the final textual part of the last event received from the deployed agent's response.

In [31]:
print(event['content']['parts'][0]['text'])

The ISS is currently over the ocean, at latitude 27.8731 and longitude -42.7579.


## 10. Clean Up: Delete the Deployed Agent

It's crucial to clean up cloud resources after you're done with them to avoid incurring unnecessary charges.
This cell calls `remote_app.delete(force=True)` to remove the Agent Engine that was deployed to Vertex AI.
- `force=True`: This option ensures the deletion proceeds even if there might be active sessions or other potential blockers. Use with caution in production environments, but it's generally safe for examples like this.

After this operation, the agent will no longer be accessible as a deployed service.

In [32]:
remote_app.delete(force=True)

2025-05-19 18:44:56,680 - vertexai.agent_engines - INFO - Delete Agent Engine backing LRO: projects/916874015322/locations/us-central1/operations/756943764936720384
2025-05-19 18:44:56,681 - vertexai.agent_engines - INFO - Agent Engine deleted. Resource name: projects/916874015322/locations/us-central1/reasoningEngines/8079396158851514368
